# Workflow Hub->LifeMonitor Interaction

In [ ]:
# required packages
import os
import requests
from pprint import pp

In [ ]:
# WhHub Settings
wfhub_username = "workflowhub"
wfhub_password = "workflowhub"
wfhub_url = "__WFHUB_INSTANCE_BASE_URL__"

In [ ]:
# LifeMonitor settings on WfHub
lm_base_url = "__LIFE_MONITOR_INSTANCE_BASE_URL__"
lm_login_url = f"{lm_base_url}/oauth2/login/seek"

In [ ]:
# create a new session
s = requests.session()
s.headers.update({
    "Content-type": "application/vnd.api+json",
    "Accept": "application/vnd.api+json",
    "Accept-Charset": "ISO-8859-1"
})
# log the user on WfHub
s.auth=requests.auth.HTTPBasicAuth(wfhub_username, wfhub_password)
print("Cookies before the first call: %r" % s.cookies)

In [ ]:
# get userinfo from Workflow Hub
user_info = s.get(os.path.join(wfhub_url, 'people/current'))
pp(user_info.json())

In [ ]:
# get workflows from Workflow Hub
workflows_response = s.get(os.path.join(wfhub_url,"workflows"))
workflows = workflows_response.json()["data"]
pp(workflows)

In [ ]:
# Pick one workflow (we assume at least one workflow is registered on Workflow Hub)
workflow_id = workflows[0]['id']
print("Workflow ID: %s" % workflow_id)

In [ ]:
# Get workflow metadata
w_response = s.get(f"{wfhub_url}/workflows/{workflow_id}")
assert w_response.status_code == 200, f"Login Error: status code {w_response.status_code} !!!"
workflow = w_response.json()["data"]
pp(workflow)



## Register a Workflow Hub workflow to LifeMonitor

We assume LifeMonitor needs to know:
1. the identity of the service which interacts with LM (e.g., a registry instance like a WfHub instance)
2. the identity of the user which triggers (directly or by delegating the registry service) the interaction from the service above

In [ ]:
# Log the user in LifeMonitor using the Workflow Hub credentials.
# This login works out of the box because WorkflowHub and LifeMonitor are sharing the same HTTP session
# and LifeMonitor has already been authorized to use the Workflow Hub user data on behalf of the user.
login_r = s.get(lm_login_url)
assert login_r.status_code == 200, "Login Error: status code {} !!!".format(login_r.status_code)

In [ ]:
# As a result of the previous step the user is also authenticated on LM and WfHub can get user data from LifeMonitor
lm_user_info_r = s.get(f"{lm_base_url}/me")
assert lm_user_info_r.status_code == 200, "Error: status code {} !!!".format(lm_user_info_r.status_code)
lm_user_info = lm_user_info_r.json()
pp(lm_user_info)

In [ ]:
# Prepare the JSON data required to post the workflow on LifeMonitor
workflow_uuid = workflow['meta']['uuid']
workflow_version = str(workflow["attributes"]["versions"][0]['version']) # pick the first version
workflow_name = workflow["attributes"]["title"]
workflow_roc_link = f'{workflow["attributes"]["content_blobs"][0]["link"]}/download'
post_data = {
    'uuid': workflow_uuid,
    'version': workflow_version,
    'name': workflow_name,
    'roc_link': workflow_roc_link
}
pp(post_data)

In [ ]:
# trigger the registration of a new workflow on LifeMonitor
response = s.post("{}/workflows".format(lm_base_url), json=post_data)
assert response.status_code == 201, "Error: status code {} !!!".format(response.status_code)
print(response.json())

In [ ]:
# get workflow with suites, test_builds and test_outputs
lm_wf_r = s.get("{}/workflows/{}/{}?test_suite=true&test_build=true&test_output=true".format(lm_base_url, workflow_uuid, workflow_version))
assert lm_wf_r.status_code == 200, f"Error: status code {lm_wf_r.status_code} !!!"
lm_workflow = lm_wf_r.json()
pp(lm_workflow)